<a href="https://colab.research.google.com/github/milazudina/ds4a_team36/blob/main/extract_skills.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [10]:
import pandas as pd
import spacy
import random
import numpy as np
import re
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')

nlp = spacy.load("en_core_web_sm")
STOPWORDS = set(stopwords.words('english'))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


# Preprocess Job Descriptions

In [52]:
# read in all the JPs that will need to have skills extracted
df_web = pd.read_csv('consolidated_df_webscrapping.csv')
df_web["Dataset"] = "web_scraped"
df_kaggle = pd.read_csv('consolidated_df_kaggle.csv')
df_kaggle["Dataset"] = "kaggle"

print(df_kaggle.columns)
print(df_web.columns)

# 302 jobs do not have a description that we will use to infer the job type, hence let's remove these from the dataframe
df_kaggle = df_kaggle[df_kaggle.Description.notnull()]
df_kaggle.reset_index(drop=True, inplace=True)
df_kaggle.shape

FileNotFoundError: ignored

In [ ]:
df_kaggle = df_kaggle.rename({'Job_title': 'Job_Title', 
                      'No_of_Stars': 'Company_Rating', 
                      'No_of_Reviews': 'Number_of_Reviews_of_the_Company',
                      'Company_Employees': 'Company_Size',
                      'Adjusted_Industry': 'Industry',
                      'Link': 'Job_URL',
                      'Queried_Salary': 'Salary'}, axis='columns')

df_kaggle["Region"] = "USA"
df_kaggle = df_kaggle.drop(columns=['Date_Since_Posted', 'Company_Industry', 'Skill', 'No_of_Skills'])
df_web = df_web.drop(columns=['Company_Founded_Year', 'Company_URL'])

df = pd.concat([df_web, df_kaggle], axis = 0, ignore_index=True)

In [ ]:
#df.Dataset[df.Dataset == 'web_scraped_uk'].shape[0]
df.Dataset.value_counts()

web_scraped    7306
kaggle         5413
Name: Dataset, dtype: int64

In [ ]:
# loop through all the job postings to extract all the nouns - for every noun put a number corresponding to where it was extracted from
# takes a couple of minutes to run

all_nouns = pd.DataFrame(columns=['Number', 'Region', 'Job_Title', 'Noun'])

for i in range(0, df.shape[0]):

  job_description = df['Description'][i].lower().replace("\n", "") # I determined experimentally that replacing 'next line' with nothing (rather than space) works best

  ## Clean the text
  job_description = job_description.replace("</b>", "").replace("</p>", "").replace("<b>", "").replace("<p>", "").replace("</li>", "").replace("</ul>", "").replace("<li>", "").replace("<ul>", "").replace("<i>", "").replace("</i>", "").replace("\r", "").replace("<div>", "").replace("h2", "").replace("h3","").replace("h1","").replace("</h1>", "").replace("</div>", "").replace("/h2", "").replace("/h3","")
  job_description = re.sub("[0-9]", "", job_description)
  job_description = job_description.replace(" the ", " ").replace(")", "").replace("(", "").replace("e.g.", "").replace("£", "").replace("$", "").replace("%", "").replace("e g", "").replace(".", " ").replace(",", " ").replace(":", "").replace(";", "").replace("?", "").replace("*", "").replace(" eg ", "").replace(">", "").replace("<", "").replace("-", " ").replace("/", " or ").replace(" a ", " ").replace(" an ", " ")
  
  text = nlp(job_description)  

  text_nouns = [chunk.text for chunk in text.noun_chunks] 
  text_nouns_clean = np.empty([len(text_nouns)], dtype='U256')

  for j in range(0,len(text_nouns)):
    # remove the stop words (It would be faster to remove it from the text directly, but I don't know if it will affect how it parses the text into noun phrases) 
    temp = [w for w in text_nouns[j].split(" ") if not w in STOPWORDS]
    temp = " ".join(temp)
    text_nouns_clean[j] = temp

  #print(text_nouns_clean)
  text_nouns_clean = text_nouns_clean[text_nouns_clean != ""]
  nouns = np.unique(text_nouns_clean)
  nouns = pd.DataFrame({'Noun':nouns})
  number = [i] * len(text_nouns)
  number = pd.DataFrame({'Number':number})
  region = [df['Region'][i]] * len(text_nouns)
  region = pd.DataFrame({'Region':region})
  job_title = [df['Job_Title'][i]] * len(text_nouns)
  job_title = pd.DataFrame({'Job_Title':job_title})

  temp = pd.concat([number, region, job_title, nouns], axis=1)

  all_nouns = pd.concat([all_nouns, temp], axis = 0)


In [ ]:
#all_nouns = pd.read_csv('all_nouns_2021-10-19.csv')
all_nouns.head(10)
all_nouns.to_csv("all_nouns_2021-10-19.csv")

In [ ]:
#keep only noun phrases with a min occurrence
#min_occurance = 2
#tokens = [k for k,c in vocab.items() if c >= min_occurane]
#print(len(tokens))

all_nouns = pd.read_csv("all_nouns_")

In [ ]:
word_occurance = all_nouns.Noun.value_counts()
print(word_occurance[0:10])
print(word_occurance[len(word_occurance)-10:len(word_occurance)])
sum(word_occurance < 2)
word_occurance[word_occurance > 2].to_csv("word_occurance_above2_2021-10-19.csv")
word_occurance_over2 = word_occurance[word_occurance > 2]

In [ ]:
word_occurance_over2 = pd.DataFrame(word_occurance_over2).rename_axis("Nouns").reset_index()
word_occurance_over2 = word_occurance_over2.rename({"Noun": "Count"}, axis = "columns")

In [ ]:
all_nouns_unique = all_nouns.drop_duplicates(subset="Noun")
print(all_nouns_unique.shape)
patterns_to_exclude = ['@','www'] 
pattern = '|'.join(patterns_to_exclude)
all_nouns_unique = all_nouns_unique[~all_nouns_unique["Noun"].str.contains(pattern, na=False)]
all_nouns_unique = all_nouns_unique[all_nouns_unique["Noun"].isin(word_occurance_over2["Nouns"]) == True]
all_nouns_unique = all_nouns_unique.rename({"Noun": "Nouns"}, axis = "columns")
all_nouns_unique = all_nouns_unique.merge(word_occurance_over2, on='Nouns')
print(all_nouns_unique.shape)
print(all_nouns_unique[0:10])
all_nouns_unique.to_csv("all_nouns_unique_2021-10-19.csv")

In [ ]:
skill_list = pd.read_csv("df_Elroy_skill_count.csv", index_col = None, header = 0)
type(skill_list["Skill"])
#all_kaggle_skills = df_kaggle_skills['Skill'].tolist()
skill_list["skill_lowerkey"] = [x.lower() for x in skill_list["Skill"]]
skill_list[0:10]

In [ ]:
all_nouns_unique.head(10)
all_nouns_unique["nouns_lowercase"] = [x.replace(' ', '') for x in all_nouns_unique["Nouns"]]
all_nouns_unique[0:10]

In [ ]:
all_nouns_unique["Skill"] = all_nouns_unique['nouns_lowercase'].isin(skill_list["skill_lowerkey"].tolist())
all_nouns_unique.loc[all_nouns_unique["Skill"] == True, "Skill"] = 1
all_nouns_unique.loc[all_nouns_unique["Skill"] == False, "Skill"] = 0
all_nouns_unique.head(10)
all_nouns_unique.to_csv("all_nouns_unique_2021-10-19_labeled_with_kaggle_skills.csv")

# LSTM + Embedding


In [ ]:
from keras.models import Sequential
from keras.layers import LSTM, Dense, Embedding, SpatialDropout1D, Flatten
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from keras.preprocessing.text import one_hot
import tensorflow as tf
from tensorflow import keras

import pandas as pd
import seaborn as sns
import numpy as np
import re

In [ ]:
# one of the files has a slightly different format, let's make it same as others
#pt3 = pt3.rename({'label': 'Label', 'confidence': 'Confidence'}, axis='columns')
#pt3 = pt3.iloc[:,0:6]
#df = pd.concat([pt1, pt2, pt3, pt4], axis = 0, ignore_index=True)

# just a general check
#sns.countplot(x='Label',data=df)

all_nouns_labeled = pd.read_csv("all_nouns_unique_2021-10-19_labeled.csv")

0    1475
1     115
3       7
2       3
Name: Skill, dtype: int64

In [ ]:
# split into input (X) and output (y) - this will be later split into the testing and training set
all_nouns = np.asarray(all_nouns_labeled.loc[:,"Nouns"])
y = all_nouns_labeled.loc[0:1600, "Skill"] # labels

# to start with, I will only use 1s and 0s
y = y.replace({2:0, 3:0})
print(y.value_counts())
y = y.tolist()

# prepare tokenizer
t = Tokenizer()
t.fit_on_texts(all_nouns)
vocab_size = len(t.word_index)+1
print(vocab_size)

# integer encode the documents
encoded_all_nouns = t.texts_to_sequences(all_nouns)
print(len(encoded_all_nouns))

list_len = [len(i) for i in encoded_all_nouns]
print(max(list_len))
#print(np.argmax(np.array(list_len)))

# pad documents to a max length
max_length = max(list_len)
padded_all_nouns = pad_sequences(encoded_all_nouns, maxlen=max_length, padding = 'post')


In [ ]:
def make_dummy_var(y):
  temp = np.zeros([len(y), 2])
  for i in range(0,len(y)):
    if y[i] == 0:
      temp[i,0] = 1
      temp[i,1] = 0
    elif y[i] == 1:
      temp[i,0] = 0
      temp[i,1] = 1
  return temp

y_onehot = make_dummy_var(y)

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(padded_all_nouns[0:len(y_onehot)], y_onehot, test_size=0.33)

In [ ]:
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(1072, 20)
(529, 20)
(1072, 2)
(529, 2)


In [ ]:
# class imbalance issue
print(1-len(y_train[y_train[:,0] == 1])/(len(y_train[y_train[:,0] == 0])+len(y_train[y_train[:,0] == 1])))
print(1-len(y_test[y_test[:,0] == 1])/(len(y_test[y_test[:,0] == 0])+len(y_test[y_test[:,0] == 1])))

0.08115671641791045
0.052930056710775


In [ ]:
# RUN THIS CELL ONLY IF USING PRETRAINED EMBEDDING (GLOVE)

embeddings_index = dict()
# can download from https://nlp.stanford.edu/projects/glove/
f = open('glove.6B.100d.txt')
for line in f:
	values = line.split()
	word = values[0]
	coefs = np.asarray(values[1:], dtype='float32')
	embeddings_index[word] = coefs
f.close()
print('Loaded %s word vectors.' % len(embeddings_index))

# create a weight matrix for words in training docs
embedding_matrix = np.zeros((vocab_size, 100))
count = 0
for word, i in t.word_index.items():
	#print(word)
	embedding_vector = embeddings_index.get(word)
	if embedding_vector is not None:
		count = count + 1
		embedding_matrix[i] = embedding_vector
print(count)
# so about 1,000 words is not found

glove_layer = Embedding(vocab_size, 100, weights=[embedding_matrix], input_length=max_length, trainable=False)

Loaded 400000 word vectors.
10580


In [ ]:
model = Sequential()
model.add(glove_layer)
#model.add(SpatialDropout1D(0.2))
#model.add(LSTM(256))
#model.add(Dense(128, activation='relu'))
#model.add(Dense(64, activation='relu'))
#model.add(Dense(32, activation='relu'))
#model.add(Dense(2, activation='softplus'))
#model.add(SpatialDropout1D(0.2))
model.add(Conv1D(filters=192, kernel_size=8, activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(Flatten())
model.add(Dense(96, activation='relu'))
model.add(Dense(48, activation='relu'))
model.add(Dense(2, activation='softmax'))

optimizer = keras.optimizers.Adam(lr=0.0001)

model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])
print(model.summary())

# add learning rate parameter

# fit the model
model.fit(X_train, y_train, epochs=50, verbose=1, batch_size=32)

# evaluate the model
loss, accuracy = model.evaluate(X_train, y_train, verbose=0)
print('Accuracy: %f' % (accuracy*100))

test_loss, test_accuracy = model.evaluate(X_test, y_test)
print('Test accuracy: %f' % (test_accuracy*100))

# first iteration (before a thourough QC) 
# Accuracy: 97.571427
# Test accuracy: 80.794168

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/optimizer_v2.py:356: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 20, 100)           1389000   
_________________________________________________________________
conv1d_3 (Conv1D)            (None, 13, 192)           153792    
_________________________________________________________________
max_pooling1d_3 (MaxPooling1 (None, 6, 192)            0         
_________________________________________________________________
flatten_3 (Flatten)          (None, 1152)              0         
_________________________________________________________________
dense_17 (Dense)             (None, 96)                110688    
_________________________________________________________________
dense_18 (Dense)             (None, 48)                4656      
_________________________________________________________________
dense_19 (Dense)             (None, 2)                

In [ ]:
predicted_y = model.predict(padded_all_nouns)

In [ ]:
# this will show the predicted and manual labels for the training set 
predicted_class=np.argmax(predicted_y[0:len(y_onehot)],axis=1)

test_nouns_with_labels = np.vstack((np.asarray(all_nouns[0:len(y_onehot)]), predicted_class, y_onehot[:,1]))

predicted_vs_labeled = pd.DataFrame(data=test_nouns_with_labels.transpose(),
                  columns=["Noun", "Predicted label", "Manual label"])



In [ ]:
predicted_vs_labeled["Predicted label"].value_counts()

0    1496
1     105
Name: Predicted label, dtype: int64

In [ ]:
print(predicted_vs_labeled[(predicted_vs_labeled['Manual label']==1) & (predicted_vs_labeled['Predicted label']==0)])
# only 11 extra skills extracted (not manually labeled)

print(sum((predicted_vs_labeled['Manual label']==0) & (predicted_vs_labeled['Predicted label']==1)))
print(sum((predicted_vs_labeled['Manual label']==1) & (predicted_vs_labeled['Predicted label']==0)))

                                Noun Predicted label Manual label
10            continuous improvement               0            1
37    information technology systems               0            1
199                           python               0            1
297                   system testing               0            1
317               azure data factory               0            1
318                 azure databricks               0            1
319                   azure services               0            1
532                         research               0            1
767                            https               0            1
865                          alteryx               0            1
958   business intelligence software               0            1
974                      mathematics               0            1
980                 power bi ability               0            1
1050           complex data analysis               0            1
1259      

In [ ]:
# this will show the predicted labels for the rest of them
predicted_class=np.argmax(predicted_y,axis=1)

test_nouns_with_labels = np.vstack((np.asarray(all_nouns_unique["Nouns"]), predicted_class))

predicted = pd.DataFrame(data=test_nouns_with_labels.transpose(),
                  columns=["Nouns", "Predicted label"])

predicted["Predicted label"].value_counts()
#print(predicted[550:600])


0    49708
1     3945
Name: Predicted label, dtype: int64

In [ ]:
print(predicted[predicted['Predicted label']==1])

                                                   Nouns Predicted label
17                                        data analytics               1
18                                       data management               1
36                                   information systems               1
53                                          presentation               1
90                                            management               1
...                                                  ...             ...
53601                       data engineering teamsmanage               1
53615  software engineering related field graduate de...               1
53621                               analytics data store               1
53626                           data processing patterns               1
53649                            hadoop  sparkexperience               1

[3945 rows x 2 columns]


In [ ]:
# extract the skills from all of them
#predicted_class=np.argmax(predicted_y,axis=1)

#test_nouns_with_labels = np.vstack((np.asarray(all_nouns), predicted_class))

#predicted = pd.DataFrame(data=test_nouns_with_labels.transpose(), columns=["Noun", "Predicted label"])

skills = predicted.loc[predicted["Predicted label"] == 1, "Nouns"]
print(skills)

skills = skills.to_list() + all_nouns_labeled.loc[all_nouns_labeled["Skill"] == 1, "Nouns"].to_list()
##skills = skills.dropna()

print(skills)

pd.DataFrame(skills).to_csv("skills_2021-10-19.csv")

17                                          data analytics
18                                         data management
36                                     information systems
53                                            presentation
90                                              management
                               ...                        
53601                         data engineering teamsmanage
53615    software engineering related field graduate de...
53621                                 analytics data store
53626                             data processing patterns
53649                              hadoop  sparkexperience
Name: Nouns, Length: 3945, dtype: object
['data analytics', 'data management', 'information systems', 'presentation', 'management', 'modelling techniques', 'data  analytics', 'enterprise data warehouse', 'postgresql', 'predictive analytics solutions', 'strong data analysis skills', 'strong technical skills', 'advanced excel skills', 'analysis', 'bus



```
# This is formatted as code
```

# Make a dataframe of Job postings & skills

['Job_Title', 'Link', 'Queried_Salary', 'Job_Type', 'Skill', 'No_of_Skills', 'Company', No_of_Reviews', 'No_of_Stars', 'Date_Since_Posted', 'Description', 'Location', ‘Company_Revenue', 'Company_Employees', 'Company_Industry']

In [256]:
word_occurance_over2 = pd.read_csv("word_occurance_above2_2021-10-19.csv")

In [303]:
# read in skills file
skills = pd.read_csv("skills_2021-10-19.csv")
skills.head(10)
skills = skills.rename({"0":"Skills"}, axis = "columns")

unique_skills = pd.DataFrame(skills.loc[:,"Skills"].unique())
# read in all nouns file
unique_skills[0:20]
unique_skills = unique_skills.rename({0:"Skills"}, axis = "columns")

temp = word_occurance_over2.rename({"Unnamed: 0":"Skills", "Noun":"Count"}, axis = "columns")
skills_with_counts = unique_skills.merge(temp, on="Skills")
skills_with_counts
# capabilities --> ability
# techniques --> skills

# check long ones specifically
skills_with_counts.to_csv("skills_with_counts_21-10-19.csv")

In [371]:
tidy_skills_with_counts = np.empty([len(skills_with_counts)], dtype='U256')

for i in range(0, skills_with_counts.shape[0]):
  temp = re.sub("^#", "", unique_skills.loc[i, "Skills"])
  temp = re.sub("^aa","",temp)
  temp = re.sub("br$", "", temp)
  #temp = re.sub("^l$","",temp)
  temp = temp.replace("[", "").replace("•", "").replace("š","s").replace("excellent", "").replace("strong", "").replace("exceptional", "").replace("exemplary","").replace("expert", "").replace("extensive", "").replace("good","").replace("great","").replace("latest","").replace("scalable", "").replace("similar","").replace("solid","").replace("sophisticated","").replace("specialized","").replace("standard", "").replace("superb", "").replace("superior", "").replace("technical", "").replace("preferred", "").replace("plus", "").replace("advanced","").strip()
  tidy_skills_with_counts[i] = temp.replace("   ", " ").replace("  ", " ").replace("analytical ", "analytics ").replace("analytic ", "analytics ").replace("methodology", "methodologies").replace("experience", "").replace("new", "").replace("ability", "").replace("proven", "").replace("capabilities", "").replace("capability", "").replace("abilities", "").replace("ability", "").replace("aaai", "artificial intelligence").strip()
  #.replace("ai ", "artificial intelligence").replace("ml ", "machine learning").replace("bi ", 'business intelligence')

tidy_skills_with_counts = pd.DataFrame(np.unique(tidy_skills_with_counts))
tidy_skills_with_counts = tidy_skills_with_counts.rename({0:"Skills"}, axis = "columns")
tidy_skills_with_counts.to_csv("tidy_skills_with_counts_2021-10-19_v2.csv")

In [372]:
tidy_skills_with_counts = tidy_skills_with_counts.merge(skills_with_counts, on="Skills", how='left')
print(tidy_skills_with_counts)

#tidy_skills_with_counts.insert(2, 'Length', 0)
for i in range(0, tidy_skills_with_counts.shape[0]):
  tidy_skills_with_counts.loc[i, 'Length'] = len(tidy_skills_with_counts.loc[i, "Skills"].split(' '))
  if len(tidy_skills_with_counts.loc[i, "Skills"]) == 0:
    tidy_skills_with_counts.loc[i, "Skills"] = "NaN"


                                Skills  Count
0                                         NaN
1     accenture digital qualifications    NaN
2                    access management    7.0
3                   account management   19.0
4                           accounting  103.0
...                                ...    ...
3664               years' data science    NaN
3665                  yield management    4.0
3666                               zfs    3.0
3667                         zookeeper    6.0
3668           zoom video conferencing    5.0

[3669 rows x 2 columns]


In [318]:
tidy_skills_with_counts = tidy_skills_with_counts[~tidy_skills_with_counts.Skills.str.contains("NaN|^l$")]
tidy_skills_with_counts.reset_index(drop=True, inplace=True)
tidy_skills_with_counts.to_csv("tidy_skills_with_counts_and_length_21-10-19.csv")

In [347]:
# before doing this, let's do some manual cleaning
tidy_skills_with_counts = pd.read_csv("tidy_skills_with_counts_and_length+exclusions_21-10-19.csv")
print(tidy_skills_with_counts.shape)
tidy_skills_with_counts = tidy_skills_with_counts[~(tidy_skills_with_counts.Exclude == 1)]
print(tidy_skills_with_counts.shape)
tidy_skills_with_counts.reset_index(inplace=True)

(3680, 5)
(3646, 5)


In [390]:
tidy_skills_with_counts_sorted = tidy_skills_with_counts.sort_values(by=["Length"], ascending = False)
tidy_skills_with_counts_sorted = tidy_skills_with_counts_sorted.reset_index()

short_skills_it1 = list()
for i in range(0, tidy_skills_with_counts_sorted.shape[1]-1):
  print(tidy_skills_with_counts_sorted.loc[i, "Skills"])
  print(len(tidy_skills_with_counts_sorted.loc[i+1:(len(tidy_skills_with_counts)-1), "Skills"]))
  #matches = tidy_skills_with_counts_sorted.loc[[phrase in tidy_skills_with_counts_sorted.loc[i, "Skills"] for phrase in tidy_skills_with_counts_sorted.loc[i+1:len(tidy_skills_with_counts)+1, "Skills"].tolist()], "Skills"]
  #matches = [phrase for phrase in tidy_skills_with_counts_sorted.loc[i, "Skills"] if phrase in tidy_skills_with_counts_sorted.loc[i+1:len(tidy_skills_with_counts)+1, "Skills"].tolist()]
  #matches = [phrase for phrase in tidy_skills_with_counts_sorted.loc[i, "Skills"] if phrase in tidy_skills_with_counts_sorted.loc[i+1:len(tidy_skills_with_counts_sorted)+1, "Skills"].tolist()]
  #pattern = '|'.join(tidy_skills_with_counts_sorted.loc[i+1:len(tidy_skills_with_counts)+1, "Skills"])
  #print(pattern)
  #matches = tidy_skills_with_counts_sorted.loc[["Skills"].str.contains(pattern), "Skills"]
  matches = [phrase for phrase in tidy_skills_with_counts_sorted.loc[i+1:len(tidy_skills_with_counts_sorted)+1, "Skills"] if phrase in tidy_skills_with_counts_sorted.loc[i, "Skills"]]
  if len(matches) == 0:
    matches = tidy_skills_with_counts_sorted.loc[i, "Skills"].split("blah")
  #print(pd.DataFrame(matches).value_counts())
  print(matches)
  print(len(matches))
  print("\n")
  #short_skills_it1 = short_skills_it1 + matches


collaborative skills team player mentality statistical analysis data mining software
3668
['statistical analysis', 'data mining', 'l', 'data', 'r', 'analysis', 'software', 'skills', 'ic', 'skill', 'c']
11


data modeling language dml data definition language ddl communication skills
3667
['data modeling', 'communication skills', 'l', 'modeling', 'data', 'mod', 'dml', 'skills', 'ic', 'communication', 'skill', 'c']
12


statistical modeling machine learning data mining concepts
3666
['statistical modeling machine', 'statistical model', 'statistical modeling', 'data mining', 'machine learning', 'l', 'modeling', 'data', 'mod', 'r', 'mac', 'ic', 'c']
13




In [375]:
# I don't want to rerun the last few cells, so will correct right here:
for i in range(0, len(short_skills_it1)):
  short_skills_it1[i] = short_skills_it1[i].replace("analyticss ", "analytics ").strip()
  short_skills_it1[i] = re.sub("analyticss$", "analytics", short_skills_it1[i])

short_skills_it1 = short_skills_it1.unique()
short_skills_it1 = pd.DataFrame(short_skills_it1)

short_skills_it1.to_csv("short_skills_it1.csv")

In [ ]:
#short_skills_it1.insert(1, "Length", 0)
short_skills_it1["Length"] = 0
short_skills_it1 = short_skills_it1.rename({0:"Skills"}, axis = "columns")
for i in range(0, short_skills_it1.shape[0]):
  short_skills_it1.loc[i, 'Length'] = len(short_skills_it1.loc[i, "Skills"].split(' '))

short_skills_it1 = short_skills_it1.sort_values(by=["Length"], ascending = False)
short_skills_it1 = short_skills_it1.reset_index()

short_skills_it2 = list()
for i in range(0, short_skills_it1.shape[0]-1):
  print(short_skills_it1.loc[i, "Skills"])
  print(len(short_skills_it1.loc[i+1:(len(tidy_skills_with_counts)-1), "Skills"]))
  #matches = tidy_skills_with_counts_sorted.loc[[phrase in tidy_skills_with_counts_sorted.loc[i, "Skills"] for phrase in tidy_skills_with_counts_sorted.loc[i+1:len(tidy_skills_with_counts)+1, "Skills"].tolist()], "Skills"]
  #matches = [phrase for phrase in tidy_skills_with_counts_sorted.loc[i, "Skills"] if phrase in tidy_skills_with_counts_sorted.loc[i+1:len(tidy_skills_with_counts)+1, "Skills"].tolist()]
  #matches = [phrase for phrase in tidy_skills_with_counts_sorted.loc[i, "Skills"] if phrase in tidy_skills_with_counts_sorted.loc[i+1:len(tidy_skills_with_counts_sorted)+1, "Skills"].tolist()]
  #pattern = '|'.join(tidy_skills_with_counts_sorted.loc[i+1:len(tidy_skills_with_counts)+1, "Skills"])
  #print(pattern)
  #matches = tidy_skills_with_counts_sorted.loc[["Skills"].str.contains(pattern), "Skills"]
  matches = [phrase for phrase in short_skills_it1.loc[i+1:len(short_skills_it1)+1, "Skills"] if phrase in short_skills_it1.loc[i, "Skills"]]
  if len(matches) == 0:
    matches = short_skills_it1.loc[i, "Skills"].split("blah")
  #print(pd.DataFrame(matches).value_counts())
  print(matches)
  print("\n")
  short_skills_it2 = short_skills_it2 + matches

In [384]:
print(len(short_skills_it2))
short_skills_it2 = pd.Series(short_skills_it2).unique()
short_skills_it2 = pd.DataFrame(short_skills_it2)

short_skills_it2.to_csv("short_skills_it2.csv")

2235


In [ ]:
all_nouns = pd.read_csv('all_nouns_2021-10-19.csv')
df.insert(16,'Extracted_skills','')

for i in range(0,df.shape[0]):

  nouns_from_job_description = all_nouns.loc[all_nouns["Number"] == i, "Noun"]
  extracted_skills = nouns_from_job_description.loc[nouns_from_job_description.isin(tidy_skills["Skills"]).values == True]
  extracted_skills_unique = extracted_skills.unique()
  print(extracted_skills_unique)
  df["Extracted_skills"][i] = extracted_skills_unique


In [ ]:
df.tail(10)

In [ ]:
df.to_csv("consolidated_df_with_skills_21-10-19.csv")